# Kafka Data Consumer

## I/ Objectifs du notebook

## II/ Mécanisme de collecte en streaming des données


In [1]:
import time
import sys
sys.path.append("../TimeSeriesTools")

In [2]:
kafka_server ='localhost:9092'
#kafka_server ='islin-hdpnod1:6667'
#kafka_server ='islin-hdplnod06:6667'

In [3]:
import kairosdb_utils
global kairosdb_server 
kairosdb_server = "http://localhost:9080"

In [4]:
import influxdb_utils
db_host= 'localhost'
port = '8086'
db_name='TimeSeriesBench'
influxdb_client = influxdb_utils.influxdb_connect(db_host, port)

Trying to connect to InfluxDB server without proxy: localhost on port: 8086
connection sucess!


In [6]:
from kafka import KafkaConsumer, TopicPartition
def consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,insert_data,convert_to_dbdata):
    print("CONNECTION KAFKA : ",kafka_server)
    print("TOPIC : ",topic)
    
    scheme = get_scheme(topic)
    consumer = KafkaConsumer(bootstrap_servers=kafka_server)

    tp = TopicPartition(topic, 0)
    consumer.assign([tp])
    consumer.seek_to_end(tp)
    lastOffset = consumer.position(tp)
    print("RANGE:",tp,lastOffset)
    consumer.seek_to_beginning(tp)
    # end
    try :
        while True:
            list_data = []
            count=0
            for message in consumer:
                msg = message.value.decode()
                count += 1
                list_data.append(msg)
                if len(list_data) == bucket_size:
                    print(count,str(len(list_data)) + " data fetched from topic : " + topic)
                    new_data = convert_to_dbdata(list_data,scheme)
                    insert_data(db_name, coll_name, new_data)
                    list_data.clear()
                if count == lastOffset:
                    print(count,str(len(list_data)) + " data fetched from topic : " + topic)
                    new_data = convert_to_dbdata(list_data,scheme)
                    insert_data(db_name, coll_name, new_data)
                    list_data.clear()
                    return

    finally:
        # Close down consumer to commit final offsets.
        consumer.close()

## III/ Mécanismes de validation et corrections des données

In [7]:
def get_scheme(topic):
    if "SmartGrid" in topic:
        scheme = {"timestamp":{'type': 'string','required': True,'empty': False},
                  "tagname":{'type': 'string','required': True,'empty': False},
                  "value":{'type': 'string','required': True,'empty': False},
                  "quality":{'type': 'string','required': True,'empty': False}}
        return scheme
    if "WindProp" in topic :
        scheme = {  "Heure":{'type': 'string','required': True,'empty': False},
                    "Temps écoulé":{'type': 'string','required': True,'empty': False},
                    '4069 state':{'type': 'string','required': True,'empty': False},
                    'Battery voltage':{'type': 'string','required': True,'empty': False},
                    'Flow SP':{'type': 'string','required': True,'empty': False},
                    'Unit Code':{'type': 'string','required': True,'empty': False},
                    'Flow M':{'type': 'string','required': True,'empty': False},
                    'Pressure1':{'type': 'string','required': True,'empty': False},
                    'Temperature1':{'type': 'string','required': True,'empty': False},
                    "Latitude":{'type': 'string','required': True,'empty': False},
                    "Longitude":{'type': 'string','required': True,'empty': False},
                    "Altitude":{'type': 'string','required': True,'empty': False},
                    "Head_Rel_True North":{'type': 'string','required': True,'empty': False},
                    "Pressure2":{'type': 'string','required': True,'empty': False},
                    "Temperature2":{'type': 'string','required': True,'empty': False},
                    "Humidity":{'type': 'string','required': True,'empty': False},
                    "MDA Wnd Dir":{'type': 'string','required': True,'empty': False},
                    "MDA Wnd Speed":{'type': 'string','required': True,'empty': False},
                    "MWD Wind Dir":{'type': 'string','required': True,'empty': False},
                    "MWD Wind Speed":{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 1':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 2':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 3':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 4':{'type': 'string','required': True,'empty': False},
                    'SPA 1':{'type': 'string','required': True,'empty': False},
                    'SPA 2':{'type': 'string','required': True,'empty': False},
                    'Cellule Photo':{'type': 'string','required': True,'empty': False},
                    'Temperature3':{'type': 'string','required': True,'empty': False},
                    'Pressure3':{'type': 'string','required': True,'empty': False},
                    'Flow MassFlow 1':{'type': 'string','required': True,'empty': False},
                    'NOTUSED Flow MassFlow 2':{'type': 'string','required': True,'empty': False},
                    'Flow':{'type': 'string','required': True,'empty': False},
                    'Humidity2':{'type': 'string','required': True,'empty': False},
                    'Test':{'type': 'string','required': True,'empty': False},
                    'Details':{'type': 'string','required': True,'empty': False},
                    'SPA 3':{'type': 'string','required': True,'empty': False},
                    'SPA 4':{'type': 'string','required': True,'empty': False},
                    "CavityPressure":{'type': 'string','required': True,'empty': False},
                    "CavityTemp":{'type': 'string','required': True,'empty': False},
                    "CH4":{'type': 'string','required': True,'empty': False},
                    "CH4_dry":{'type': 'string','required': True,'empty': False},
                    "C2H6":{'type': 'string','required': True,'empty': False},
                    "C2H6_dry":{'type': 'string','required': True,'empty': False},
                    "13CH4":{'type': 'string','required': True,'empty': False},
                    "H2O":{'type': 'string','required': True,'empty': False},
                    "CO2":{'type': 'string','required': True,'empty': False},
                    "C2C1Ratio":{'type': 'string','required': True,'empty': False},
                    "Delta_iCH4_Raw":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_30s":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_2min":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_5min":{'type': 'string','required': True,'empty': False} }
        return scheme

In [8]:
def clean_data(data,scheme):
    from cerberus import Validator
    v = Validator(scheme)
    for index,item in enumerate(data,start=0):
        res = v.validate(item)
        if (res == False):
            print("corrupt data in line :",index,", error : ",v.errors)
            del data[index]

## IV/ Mécanismes de mise en base des données

In [9]:
import mongodb_utils
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
mongodb_client = mongodb_utils.mongodb_connect(db_host, port)

Trying to connect to MongoDB server: localhost on port: 27018


### A/ Mise en base du schéma des données

In [10]:
def str_to_unix(date):
    from datetime import datetime
    dt = datetime.strptime(date, '%d/%m/%Y %H:%M:%S')
    epoch = datetime.utcfromtimestamp(0)
    return int((dt - epoch).total_seconds()) * 1000

In [11]:
def type_convert(data):
    data = data.replace(',', '.')
    time = str_to_unix(data.split(";", 1)[0].split(":", 1)[1])
    tags = data.split(";", 2)[1]
    data_treated = [tags,[time, data.split(";", 2)[2]]]
    return data_treated

In [12]:
def mongodb_convert_to_dbdata(list_data,scheme):
    col= [k for k in scheme.keys()]
    new_dict=[]
    
    t0 = time.process_time()
    for item in list_data:
        info=item.split(":",1)[1].split(";")
        dict_item ={col[i]:info[i] for i in range(len(col))}
        new_dict.append(dict_item)
    #clean_data(new_dict,scheme)
    t1 = time.process_time()
    #print('.. %f seconds for validate' % (t1 - t0))
    
    return new_dict

In [13]:
def db_convert_to_dbdata(list_msg,scheme):
    from cerberus import Validator
    col = [k for k in scheme.keys()]
    #v = Validator(scheme)
    db_data = []
    t0 = time.process_time()
    for index,item in enumerate(list_msg):
        #info = item.split(":", 1)[1].split(";")
        #dict_item = {col[i]: info[i] for i in range(len(col))}
        #res = v.validate(dict_item)
        res = True
        if res == True:
            new_data = type_convert(item)
            db_data.append(new_data)
        else:
            print("corrupt data in line :",index,", error : ",v.errors)
    t1 = time.process_time()
    #print('.. %f seconds for validate' % (t1 - t0))
    return db_data

In [14]:
def mongodb_insert_data(db_name, coll_name, list_data):
    mongodb_utils.insert_many_docs(mongodb_client,db_name, coll_name,list_data)    
    return len(list_data)

In [15]:
def kairosdb_insert_data(db_name, coll_name, list_data):
    tags_data_list = {}
    for d in list_data:
        if d[0] in tags_data_list.keys():
            tags_data_list[d[0]].append(d[1])
        else:
            tags_data_list[d[0]] = [d[1]]
    
    tag_keys = ['Buiding','Device','Measure']
    for i,(k,v) in enumerate(tags_data_list.items()):
        tag_values = k.split(".")
        tags = {tag_keys[i]:tag_values[i] for i in range(len(tag_keys))}
        kairosdb_utils.insert_many_docs_with_tags(kairosdb_server,db_name, coll_name,tags,v)    
    return len(list_data)

In [16]:
def influxdb_insert_data(db_name, coll_name, list_data):
    tags_data_list = {}
    for d in list_data:
        if d[0] in tags_data_list.keys():
            tags_data_list[d[0]].append(d[1])
        else:
            tags_data_list[d[0]] = [d[1]]
    
    tag_keys = ['Buiding','Device','Measure']
    for i,(k,v) in enumerate(tags_data_list.items()):
        tag_values = k.split(".")
        tags = {tag_keys[i]:tag_values[i] for i in range(len(tag_keys))}
        influxdb_utils.insert_many_docs_with_tags(influxdb_client,db_name, coll_name,tags,v)    
    return len(list_data)

#### 1) Cas test SmartGrid

In [17]:
topic = 'SmartGridDataCryolite20190101OneMonthTopic'
scheme = get_scheme(topic)

In [18]:
db=mongodb_client[db_name]
db.list_collection_names()

['WindPropLacqOneDayP25',
 'schemes',
 'SmartGridCryolite20190101OneMonth',
 'SmartGridCryolite20190101OneMonthV2',
 'WindPropLacqOneDay',
 'WindPropLacqOneDayV2',
 'SmartGridCryolite20190101OneMonthV1']

In [ ]:
scheme_collection = db['schemes']

In [ ]:
scheme_id = scheme_collection.insert_one(scheme)

In [ ]:
scheme_id

In [21]:
coll_name='SmartGridCryolite20190101OneMonthV10000'

In [23]:
%%time
bucket_size = 10000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,mongodb_insert_data,mongodb_convert_to_dbdata)

CONNECTION KAFKA :  localhost:9092
TOPIC :  SmartGridDataCryolite20190101OneMonthTopic
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
10000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
20000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
30000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
40000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
50000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
60000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
70000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
80000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
90000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
100000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
110000 1

In [22]:
%%time
bucket_size = 10000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,kairosdb_insert_data,db_convert_to_dbdata)

CONNECTION KAFKA :  localhost:9092
TOPIC :  SmartGridDataCryolite20190101OneMonthTopic
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
10000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
20000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
30000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
40000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
50000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
60000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
70000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
80000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
90000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
100000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
110000 1

In [20]:
%%time
bucket_size = 10000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,influxdb_insert_data,db_convert_to_dbdata)

CONNECTION KAFKA :  localhost:9092
TOPIC :  SmartGridDataCryolite20190101OneMonthTopic
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
10000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
10000 documents have been inserted
20000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
10000 documents have been inserted
30000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
7584 documents have been inserted
2416 documents have been inserted
40000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
10000 documents have been inserted
50000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
10000 documents have been inserted
60000 10000 data fetched from topic : SmartGridDataCryolite20190101OneMonthTopic
1652 documents have been inserted
8348 documents have been inserted
70000 10000 data fetched from topic : SmartGridDataCryoli

### 2) Cas test WindProp

In [29]:
def type_convert(data):
    data = data.replace(',', '.')
    time = str_to_unix(data.split(";", 1)[0].split(":", 1)[1])
    data_treated = [time, data.split(";", 1)[1]]
    return data_treated

In [30]:
def db_convert_to_dbdata(list_msg,scheme):
    from cerberus import Validator
    col = [k for k in scheme.keys()]
    #v = Validator(scheme)
    db_data = []
    t0 = time.process_time()
    for index,item in enumerate(list_msg):
        #info = item.split(":", 1)[1].split(";")
        #dict_item = {col[i]: info[i] for i in range(len(col))}
        #res = v.validate(dict_item)
        res = True
        if res == True:
            new_data = type_convert(item)
            db_data.append(new_data)
        else:
            print("corrupt data in line :",index,", error : ",v.errors)
    t1 = time.process_time()
    #print('.. %f seconds for validate' % (t1 - t0))
    return db_data

In [31]:
def kairosdb_insert_data(db_name, coll_name, list_data):
    kairosdb_utils.insert_many_docs(kairosdb_server,db_name, coll_name,list_data)    
    return len(list_data)

In [32]:
def influxdb_insert_data(db_name, coll_name, list_data):
    influxdb_utils.insert_many_docs(influxdb_client,db_name, coll_name,list_data)    
    return len(list_data)

In [33]:
topic = 'WindPropDataLacqOneDayP25NewTopic'

In [34]:
scheme = get_scheme(topic)

In [ ]:
db = mongodb_client[db_name]
scheme_collection = db['schemes']

In [ ]:
scheme_id = scheme_collection.insert_one(scheme)

In [ ]:
scheme_id

In [35]:
coll_name='WindPropLacqOneDayP25-B1000'

In [27]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,mongodb_insert_data,mongodb_convert_to_dbdata)

CONNECTION KAFKA :  localhost:9092
TOPIC :  WindPropDataLacqOneDayP25NewTopic
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
1000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
2000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
3000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
4000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
5000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
6000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
7000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
7288 288 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
CPU times: user 474 ms, sys: 13 ms, total: 487 ms
Wall time: 951 ms


In [36]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,kairosdb_insert_data,db_convert_to_dbdata)

CONNECTION KAFKA :  localhost:9092
TOPIC :  WindPropDataLacqOneDayP25NewTopic
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
1000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
.. 0.000150 seconds for create query
.. 0.006846 seconds for posting query
insertion many doc: 	204 (status code)
2000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
.. 0.000105 seconds for create query
.. 0.005951 seconds for posting query
insertion many doc: 	204 (status code)
3000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
.. 0.000170 seconds for create query
.. 0.005766 seconds for posting query
insertion many doc: 	204 (status code)
4000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
.. 0.000188 seconds for create query
.. 0.005130 seconds for posting query
insertion many doc: 	204 (status code)
5000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
.. 0.000166 seconds for create qu

In [37]:
%%time
bucket_size = 1000
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,influxdb_insert_data,db_convert_to_dbdata)

CONNECTION KAFKA :  localhost:9092
TOPIC :  WindPropDataLacqOneDayP25NewTopic
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
1000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
2000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
3000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
4000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
5000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
6000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
7000 1000 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
1000 documents have been inserted
7288 288 data fetched from topic : WindPropDataLacqOneDayP25NewTopic
288 documents have been inserted
CPU times

### 3) Cas test Pollution